In [1]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import pyodbc
from sklearn.feature_extraction.text import TfidfVectorizer 

In [ ]:
nltk.download('punkt')

In [ ]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [ ]:
articlesContent = execQuery("""Select content
from article""")

In [2]:
#connection = psycopg2.connect(user = "postgres",
#                                      password = "root",
#                                      host = "localhost",
#                                      port = "5432",
#                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random) 
filepath = '/home/daniel/OneDrive/KUuni/DataScience/Python/small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

In [3]:
df_content = df['content'].str.lower()
series_content_tokens = df_content.apply(nltk.word_tokenize)

In [4]:
series_content_tokens = series_content_tokens.explode()
df_content_tokens = series_content_tokens.to_frame()
df_content_tokens = df_content_tokens.reset_index().rename(columns={'index':'content_index'})
series_content_tokens = df_content_tokens.groupby('content_index')['content'].value_counts(normalize=True)

In [5]:
series_content_tokens

content_index  content     
0              the             0.084715
               .               0.040516
               of              0.036832
               to              0.029466
               in              0.025783
                                 ...   
249            unacceptable    0.007752
               wa              0.007752
               wednesday       0.007752
               will            0.007752
               yangon          0.007752
Name: content, Length: 76994, dtype: float64

In [6]:
# create TF
TF = series_content_tokens.unstack(level=0)
TF.head()

content_index,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
content,,,,,,,,,,,,,,,,,,,,,
!,0.007366,0.008969,NaN,NaN,NaN,NaN,0.026316,NaN,0.003388,NaN,...,0.004484,NaN,NaN,NaN,NaN,0.001922,0.002985,NaN,0.002016,NaN
#,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$,0.001842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014037,NaN,...,NaN,NaN,0.003247,NaN,NaN,0.001281,NaN,NaN,NaN,NaN
%,NaN,0.004484,NaN,NaN,NaN,NaN,NaN,NaN,0.003872,NaN,...,NaN,NaN,NaN,NaN,NaN,0.004484,NaN,NaN,NaN,NaN
&,NaN,0.004484,0.001912,NaN,NaN,NaN,NaN,NaN,0.000968,NaN,...,0.002242,NaN,NaN,NaN,NaN,0.000641,0.002985,NaN,NaN,NaN


In [341]:
a = TF[TF > 0] / np.log(1 + len(TF))
a.iloc[0,0]

0.0007521289603870707

In [388]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df['sent'])

In [1]:
feature_names = tfidf.get_feature_names()
corpus_index = [n for n in corpus]
import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
print(df)

NameError: name 'tfidf' is not defined

In [111]:
import numpy as np

df_1 = pd.DataFrame({'sent': 
                    ['This This is is first', 
                     'This is the second sentence',
                     'This the third sentence']})

# Tokenize and generate count vectors
word_vec = df_1.sent.str.split().apply(pd.value_counts).fillna(0)

# Compute term frequencies
tf = word_vec.divide(np.sum(word_vec, axis='columns'), axis='index')

# Compute inverse document frequencies
idf = np.log(len(tf) / word_vec[word_vec > 0].count()) 

# Compute TF-IDF vectors
tfidf = np.multiply(tf, idf.to_frame().T)

print(word_vec[word_vec > 0].count())

This        3
is          2
first       1
second      1
the         2
sentence    2
third       1
dtype: int64


In [70]:
df = pd.DataFrame({'docId': [1,2,3], 
               'sent': ['This is the first sentence', 
                        'This is the second sentence',
                        'This is the third sentence']})

# use 'value_counts' to get counts of items in list
tf = df.sent.apply(pd.value_counts).fillna(0) 
# tf = df.sent.str.split().apply(pd.value_counts).fillna(0)

# add one to numerator and denominator just incase a term isn't in any document
# maximum value is log(N) and minimum value is zero
tf

,This is the first sentence,This is the second sentence,This is the third sentence
0,26.0,0.0,0.0
1,0.0,27.0,0.0
2,0.0,0.0,26.0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
df = pd.DataFrame({'docId': [1,2,3], 
               'sent': ['This is the first sentence','This is the second sentence', 'This is the third sentence']})

v = TfidfVectorizer()
x = v.fit_transform(df['sent'])
 
type(x)

scipy.sparse.csr.csr_matrix